In [4]:
import torch
from torch.nn import functional as F

def conv_forward_naive(x, w, b, conv_param):
    """
    A naive Python implementation of a convolution.
    The input consists of an image tensor with height H and
    width W. We convolve each input with a filter F, where the filter
    has height HH and width WW.
    Input:
    - x: Input data of shape (B , C ,H, W)
    - w: Filter weights of shape (C_out , C , HH, WW)
    - b: Bias for filter
    - conv_param: A dictionary with the following keys:
      - 'stride': The number of pixels between adjacent receptive fields in the
        horizontal and vertical directions.
      - 'pad': The number of pixels that will be used to zero-pad the input. 
        
    During padding, 'pad' zeros should be placed symmetrically (i.e equally on both sides)
    along the height and width axes of the input. Be careful not to modfiy the original
    input x directly.
    Returns an array.
    - out: Output data, of shape (H', W') where H' and W' are given by
      H' = 1 + (H + 2 * pad - HH) / stride
      W' = 1 + (W + 2 * pad - WW) / stride
    """

    
    pad , stride = conv_param['pad'] , conv_param['stride']
    
    B , input_channels , H , W = x.shape
    C_out , filter_channels, filter_height , filter_width =w.shape
    stride , pad = conv_param['stride'] , conv_param['pad']
    assert input_channels == filter_channels , 'the input and the filters must have the same number of channels'

    # Check dimensions.
    assert (W + 2 * pad - filter_width) % stride == 0, 'width does not work'
    assert (H + 2 * pad - filter_height) % stride == 0, 'height does not work'

    H_o  = 1 + int((W + 2 * pad - filter_height) / stride)
    W_o= 1 + int((W + 2 * pad - filter_height) / stride)

    out = torch.randn(B,C_out , H_o , W_o)
    # print('output shape ',out.shape)
    ###########################################################################
    # TODO: Implement the convolutional forward pass.                         #
    # Hint: you can use the function torch.nn.functional.pad for padding.     #
    padding = (
        pad,pad,   
        pad,pad,
    )
    x_pad = F.pad(x ,padding)

    def convolve(x,k):
      conv = torch.sum( x * k , (1,2,3))
      return conv
    for e in range(B):
      for i in range(H_o):
        for j in range(W_o):
          # print('i ,j ',i , j)
          k=filter_height
          # print('k= ' , k)
          
          out[e, : ,i,j]= convolve( x_pad[e , : , i*stride:i*stride+k , j*stride:j*stride+k] , w) + b #all batch , all channels , window , window

    return out
    ###########################################################################
    


In [5]:

# Make convolution module.
w_shape = (3, 3, 4, 4)
w = torch.linspace(-0.2, 0.3, steps=torch.prod(torch.tensor(w_shape))).reshape(w_shape)
b = torch.linspace(-0.1, 0.2, steps=3)

# Compute output of module and compare against reference values.
x_shape = (2, 3, 4, 4)
x = torch.linspace(-0.1, 0.5, steps=torch.prod(torch.tensor(x_shape))).reshape(x_shape)
out = conv_forward_naive(x, w, b, {'stride': 2, 'pad': 1})


correct_out = torch.tensor([[[[-0.08759809, -0.10987781],
                              [-0.18387192, -0.2109216 ]],
                             [[ 0.21027089,  0.21661097],
                              [ 0.22847626,  0.23004637]],
                             [[ 0.50813986,  0.54309974],
                              [ 0.64082444,  0.67101435]]],
                            [[[-0.98053589, -1.03143541],
                              [-1.19128892, -1.24695841]],
                             [[ 0.69108355,  0.66880383],
                              [ 0.59480972,  0.56776003]],
                             [[ 2.36270298,  2.36904306],
                              [ 2.38090835,  2.38247847]]]])

print('---'*10)
print('out shape' , out.shape)
print('correct_out shape' , correct_out.shape)


print('---'*10)

# Compare your output to ours; difference should be around e-8
print('Testing conv_forward_naive')
rel_error = ((out - correct_out) / (out + correct_out + 1e-6)).mean()
print('difference: ', rel_error)
if abs(rel_error) < 1e-6:
    print('Nice work! Your implementation of a convolution layer works correctly.')
else:
    print('Something is wrong. The output was expected to be \n{} \nbut it was \n{}'.format(correct_out, out))

------------------------------
out shape torch.Size([2, 3, 2, 2])
correct_out shape torch.Size([2, 3, 2, 2])
------------------------------
Testing conv_forward_naive
difference:  tensor(2.8497e-08)
Nice work! Your implementation of a convolution layer works correctly.
